# Marko Zlatic ISI/Esri Student Competition 2023 Python Code

**This Python code was created to uphold requirements for the ISI/Esri Student Competition 2023.**

The Python code below is used to output a csv file containing all the locations for predicted fires and their respected causes from 2022 to 2102. XGBoost, as well as, Sklearn KMeans and DBSCAN were the primary machine learning algorithms used to dictate the predicted fire cause and at what location. An XGBoost regressor was used to carry out the final computation and an R-Squared score dictated the accuracy of the model.

The source dataset used to produce the predictions is from the Canadian Wildland Fire Information System (CWFIS) Fire History Data found [here]("https://cwfis.cfs.nrcan.gc.ca/datamart").

The main feature layer powering this web application can be found [here]("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/Forest_Fire_Prediction_in_Canada_8_Years/FeatureServer").

The web application hosting the results can be found in [GitHub]("https://github.com/mzlatic1/MZ_ISI_Esri_Student_Competition/blob/gh-pages/isi_cmp_html.html") for the source code and can also be viewed from the JSbin hosted link [here]("https://output.jsbin.com/lohawuv").

Author Info:<br/>
Name: Marko Zlatic<br/>
Date: May 28, 2023<br/>
Purpose: ISI/Esri Student Competition 2023<br/>
Student Status: Graduate<br/>
Program: MSc. Geographic Information Systems<br/>
University: Johns Hopkins University


In [1]:
# Import all necessary packages and modules

from sklearn.preprocessing import FunctionTransformer, LabelEncoder
from sklearn import model_selection, metrics
from sklearn.cluster import KMeans, DBSCAN
from xgboost import XGBRegressor
import geopandas as gpd
import pandas as pd
import numpy as np
import os


**Step 1: Data Manipulation.** ArcGIS Pro was used first to prep the initial dataset (see StoryMaps). Geopandas was used to read the imported feature class, however, arcpy's TableToExcel_management() function could have been used export the features then read the spreadsheet via Pandas. Briefly mentioned, ArcGIS Pro was used to first prep the shapefile and exporting to a local GDB. The dataframe is then manipulated to ensure it is adequate to run through the machine learning model.

In [2]:
# Load data
gdb = os.path.join(os.path.split(os.getcwd())[0], 'ISI_Esri_Competition.gdb')

fire_points = gpd.read_file(gdb, layer='NFDB_point_20220901_CLEAN')


In [3]:
fire_points.head()

,FID,SRC_AGENCY,FIRE_ID,FIRENAME,LATITUDE,LONGITUDE,YEAR,MONTH,DAY,REP_DATE,...,CFS_NOTE1,CFS_NOTE2,ACQ_DATE,SRC_AGY2,ECOZONE,ECOZ_REF,ECOZ_NAME,ECOZ_NOM,fire_cause,geometry
0,0,BC,1953-G00041,NaN,59.963,-128.172,1953,5,26,1953-05-26T00:00:01+00:00,...,NaN,NaN,2020-05-05T00:00:00+00:00,BC,12,12,Boreal Cordillera,CordillCre boreale,1,POINT (-1720729.243 1659436.548)
1,1,BC,1950-R00028,NaN,59.318,-132.172,1950,6,22,1950-06-22T00:00:01+00:00,...,NaN,NaN,2020-05-05T00:00:00+00:00,BC,12,12,Boreal Cordillera,CordillCre boreale,3,POINT (-1944085.814 1715215.898)
2,2,BC,1950-G00026,NaN,59.876,-131.922,1950,6,4,1950-06-04T00:00:01+00:00,...,NaN,NaN,2020-05-05T00:00:00+00:00,BC,12,12,Boreal Cordillera,CordillCre boreale,1,POINT (-1899364.262 1758150.284)
3,3,BC,1951-R00097,NaN,59.760,-132.808,1951,7,15,1951-07-15T00:00:01+00:00,...,NaN,NaN,2020-05-05T00:00:00+00:00,BC,12,12,Boreal Cordillera,CordillCre boreale,1,POINT (-1946555.179 1774478.027)
4,4,BC,1952-G00116,NaN,59.434,-126.172,1952,6,12,1952-06-12T00:00:01+00:00,...,NaN,NaN,2020-05-05T00:00:00+00:00,BC,12,12,Boreal Cordillera,CordillCre boreale,1,POINT (-1652703.113 1556258.550)


In [4]:
# remove unnecessary fields
fields_2_keep = ['SRC_AGENCY', 'LATITUDE', 'LONGITUDE', 'YEAR', 'MONTH', 'DAY', 'DECADE', 'SIZE_HA', 'CAUSE', 'ECOZONE', 'ECOZ_REF', 'ECOZ_NAME', 'ECOZ_NOM']
fire_points.drop(columns=[col for col in list(fire_points.columns) if col not in fields_2_keep], axis=1, inplace=True)


In [5]:
# removed data points that would hinder the model to produce robust results
fire_points = fire_points[fire_points.notnull().all(1)]
fire_points.query("SIZE_HA > 0 & YEAR >= 1950", inplace=True)


In [6]:
fire_points.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 381363 entries, 0 to 419843
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   SRC_AGENCY  381363 non-null  object 
 1   LATITUDE    381363 non-null  float64
 2   LONGITUDE   381363 non-null  float64
 3   YEAR        381363 non-null  int64  
 4   MONTH       381363 non-null  int64  
 5   DAY         381363 non-null  int64  
 6   DECADE      381363 non-null  object 
 7   SIZE_HA     381363 non-null  float64
 8   CAUSE       381363 non-null  object 
 9   ECOZONE     381363 non-null  int64  
 10  ECOZ_REF    381363 non-null  object 
 11  ECOZ_NAME   381363 non-null  object 
 12  ECOZ_NOM    381363 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 40.7+ MB


The sin_transformer() was derived from a blog post created from NVIDIA's Eryk Lewinson found here: [https://developer.nvidia.com/blog/three-approaches-to-encoding-time-information-as-features-for-ml-models/ Retrieved May 28, 2023]("https://developer.nvidia.com/blog/three-approaches-to-encoding-time-information-as-features-for-ml-models/"). Transforming the month and day integers promotes greater accuracy for the model as months 12 and 1 (for example) are more closely related when taking the sin of the respected integers; where as, literally speaking, the numbers 1 and 12 arent so close together, when comparing the months of January and December; they're 'right next to each other'.

In [7]:
# date transformer function
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


In [8]:
# format dates for model
date_fields = {'MONTH': 12, 'DAY': 365}
for field in date_fields:
    col_name = field + '_SIN'
    fire_points[col_name] = sin_transformer(date_fields[field]).fit_transform(fire_points[field])

fire_points.drop(columns=list(date_fields.keys()), axis=1, inplace=True)

In [9]:
fire_points.head()

,SRC_AGENCY,LATITUDE,LONGITUDE,YEAR,DECADE,SIZE_HA,CAUSE,ECOZONE,ECOZ_REF,ECOZ_NAME,ECOZ_NOM,MONTH_SIN,DAY_SIN
0,BC,59.963,-128.172,1953,1950-1959,8.0,H,12,12,Boreal Cordillera,CordillCre boreale,5.000000e-01,0.432776
1,BC,59.318,-132.172,1950,1950-1959,8.0,L,12,12,Boreal Cordillera,CordillCre boreale,1.224647e-16,0.369725
2,BC,59.876,-131.922,1950,1950-1959,12949.9,H,12,12,Boreal Cordillera,CordillCre boreale,1.224647e-16,0.068802
3,BC,59.760,-132.808,1951,1950-1959,241.1,H,12,12,Boreal Cordillera,CordillCre boreale,-5.000000e-01,0.255353
4,BC,59.434,-126.172,1952,1950-1959,1.2,H,12,12,Boreal Cordillera,CordillCre boreale,1.224647e-16,0.205104


In [10]:
# log the SIZE_HA field to normalize values
fire_points['SIZE_HA_LOG'] = np.log(fire_points['SIZE_HA'])
fire_points.drop(columns=['SIZE_HA'], axis=1, inplace=True)


In [11]:
fire_points.head()

,SRC_AGENCY,LATITUDE,LONGITUDE,YEAR,DECADE,CAUSE,ECOZONE,ECOZ_REF,ECOZ_NAME,ECOZ_NOM,MONTH_SIN,DAY_SIN,SIZE_HA_LOG
0,BC,59.963,-128.172,1953,1950-1959,H,12,12,Boreal Cordillera,CordillCre boreale,5.000000e-01,0.432776,2.079442
1,BC,59.318,-132.172,1950,1950-1959,L,12,12,Boreal Cordillera,CordillCre boreale,1.224647e-16,0.369725,2.079442
2,BC,59.876,-131.922,1950,1950-1959,H,12,12,Boreal Cordillera,CordillCre boreale,1.224647e-16,0.068802,9.468843
3,BC,59.760,-132.808,1951,1950-1959,H,12,12,Boreal Cordillera,CordillCre boreale,-5.000000e-01,0.255353,5.485212
4,BC,59.434,-126.172,1952,1950-1959,H,12,12,Boreal Cordillera,CordillCre boreale,1.224647e-16,0.205104,0.182322


In [12]:
print(len(fire_points.index)) # number of rows
fire_points.isna().sum() # check for any remaining null values

381363


SRC_AGENCY     0
LATITUDE       0
LONGITUDE      0
YEAR           0
DECADE         0
CAUSE          0
ECOZONE        0
ECOZ_REF       0
ECOZ_NAME      0
ECOZ_NOM       0
MONTH_SIN      0
DAY_SIN        0
SIZE_HA_LOG    0
dtype: int64

In [13]:
# transform the remaining character-based fields
for column in list(fire_points.columns):
    if fire_points[column].dtype == object and column:
        fire_points[column] = LabelEncoder().fit_transform(fire_points[column]) + 1


**Step 2: Running the Model.** Now that the dataframe is prepped, the custom machine learning model will now take the dataframe and first run it through the DBSCAN algorithm; this will indicate the most optimal number of clusters to use given the epsilon (EPS) variable (ie the maximum search distance). Then, with the number of optimal clusters derived, it then runs through the KMeans algorithm that categorizes each of the input training points and assigns a cluster number; this is then joined back to the main dataframe. Now with each row having a cluster ID, the dataframe is then split into training and validation subsets. Once when the dataframe is split, the training data is then input for the XGBoost regressor algorithm. Once when the regressor completes its first iteration, it's then used to predict the validation data; then becomes input for the R-Squared metric generator. If the R-Squared is less than 0.99, the EPS variable is subtracted by 0.1 until the desired R-Squared is achieved (r^2 >= 0.99). Once when the desired R-Squared is achieved, the primary dataframe is then ran through the model to produce the predictions for the next 8 decades.

In [14]:
subset_4_clustering = fire_points[['LATITUDE', 'LONGITUDE', 'CAUSE']]

In [15]:
eps = 0.2 # originally the eps was first set to 0.9, however due to runtime contains and previous QC, the source code starts at 0.2.
while True:
    print('Testing with eps:', eps)

    # First, the main dataframe is run through the DBSCAN algorithm.
    db_scan = DBSCAN(eps, min_samples=50, n_jobs=-1).fit(subset_4_clustering)
    prediction = db_scan.fit_predict(subset_4_clustering)

    # Then the KMeans algorithm is then run using the number of clusters from the DBSCAN results.
    num_clusters = len(np.unique(prediction))
    k_means = KMeans(n_clusters=num_clusters, n_init='auto', max_iter=300, random_state=42)
    k_means.fit(subset_4_clustering)

    cluster_pred = k_means.predict(subset_4_clustering)

    if 'cluster_id' in list(fire_points.columns): # if the iteration needs to repeat itself, it removes the pre-existing cluster id results.
        fire_points.drop(columns='cluster_id', axis=1, inplace=True)

    # Resulting cluster IDs are then joined to the main dataframe.
    cluster_id = pd.DataFrame(cluster_pred, columns=['cluster_id']).reset_index().rename(columns={'index': 'join_field'})
    fire_points = fire_points.reset_index().rename(columns={'index': 'join_field'}).merge(cluster_id, on='join_field').drop(columns='join_field', axis=1)

    # The main dataframe is then reorganized, and the training and validation subsets are executed.
    reordered_fields = ['SRC_AGENCY', 'YEAR', 'MONTH_SIN', 'DAY_SIN', 'DECADE', 'ECOZONE', 'ECOZ_REF', 'ECOZ_NAME', 'ECOZ_NOM', 'SIZE_HA_LOG', 'cluster_id', 'CAUSE', 'LONGITUDE', 'LATITUDE']
    fire_points = fire_points[reordered_fields]

    x = fire_points.iloc[:, :-3]
    y = fire_points.iloc[:, -3:]

    xtrain, xvalid, ytrain, yvalid = model_selection.train_test_split(x, y, test_size=0.2, random_state=42)

    # XGBoost Regressor
    params = {
        'n_estimators': 500,
        'max_leaves': 0,
        'n_jobs': -1,
        'random_state': 42,
        'gpu_id': 0,
        'predictor': 'gpu_predictor'
    }

    xg_regress = XGBRegressor(**params) # Prep model with parameters
    xg_regress.fit(xtrain, ytrain) # Train the regressor model
    ypred = xg_regress.predict(xvalid) # Establish predictions with validation subset

    r2 = metrics.r2_score(yvalid, ypred) # Generates an R-Squared Score
    if r2 >= 0.99:
        print('R2 Score:', r2, 'Running model with actual dataset.')
        ypred = xg_regress.predict(x) # reruns predictions if desired R-Squared achieved.
        break
    else:
        eps -= 0.1


Testing with eps: 0.2
Testing with eps: 0.1
R2 Score: 0.9917404845211535 Running model with actual dataset.


**Step 3: Final Clean and Export Results.** Although robust, the XGBoost regressor might predict values that are invalid when transcribing them back into the coded values. For example, the fire cause field was transformed and ranges from 1 through 6. If there's a predicted value that greater than 6.5, then the categorical value is 7 (rounded to the nearest whole number), which is an invalid category.

In [16]:
print('Exporting results...')
initial_row_count = len(ypred)
df_ypred = pd.DataFrame(ypred, columns=["fire_cause", "longitude", "latitude"]).query("0.5 <= fire_cause < 6.5") # query to remove invalid predictions
final_row_count = len(df_ypred.index)
print((initial_row_count - final_row_count), 'rows removed.')
df_ypred['fire_cause'] = df_ypred.apply(lambda row: round(row['fire_cause']), axis=1)

df_ypred.to_csv(os.path.join(os.path.split(os.getcwd())[0], 'fire_pred.csv'), index=False)
print('Script complete!')

Exporting results...
4 rows removed.
Script complete!


**Step 4: Using ArcGIS Pro to import the csv file and conduct final processing before publishing results as a feature layer.** The exported csv file is then imported as a feature class using the XYTableToPoint_management() arcpy geoprocessing tool. The transformed fire cause field (use to be text values, now has values ranging from 1 to 6), is then re-transformed using the arcpy.da.UpdateCursor() function (see StoryMap for more detail).